## Lineaire en Polynomiale Regressie voor Pompoenprijzen - Les 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographic door Dasani Madipalli</figcaption>


<!--![Infographic door Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Introductie

Tot nu toe heb je onderzocht wat regressie is met voorbeeldgegevens verzameld uit de pompoenprijsdataset die we gedurende deze les zullen gebruiken. Je hebt het ook gevisualiseerd met behulp van `ggplot2`. 💪

Nu ben je klaar om dieper in te gaan op regressie voor machine learning. In deze les leer je meer over twee soorten regressie: *basis lineaire regressie* en *polynomiale regressie*, samen met enkele wiskundige principes die aan deze technieken ten grondslag liggen.

> Door deze hele cursus gaan we uit van minimale wiskundige kennis en proberen we het toegankelijk te maken voor studenten uit andere vakgebieden. Let daarom op notities, 🧮 wiskundige aanwijzingen, diagrammen en andere leermiddelen om het begrip te vergemakkelijken.

#### Voorbereiding

Ter herinnering: je laadt deze gegevens in om er vragen over te stellen.

-   Wanneer is de beste tijd om pompoenen te kopen?

-   Welke prijs kan ik verwachten voor een kist met miniatuurpompoenen?

-   Moet ik ze kopen in halve schepelmanden of in een 1 1/9 schepeldoos? Laten we verder graven in deze gegevens.

In de vorige les heb je een `tibble` (een moderne herinterpretatie van het data frame) gemaakt en deze gevuld met een deel van de oorspronkelijke dataset, waarbij je de prijzen standaardiseerde per schepel. Door dat te doen, kon je echter slechts ongeveer 400 datapunten verzamelen en alleen voor de herfstmaanden. Misschien kunnen we wat meer details over de aard van de gegevens krijgen door ze verder op te schonen? We zullen zien... 🕵️‍♀️

Voor deze taak hebben we de volgende pakketten nodig:

-   `tidyverse`: Het [tidyverse](https://www.tidyverse.org/) is een [verzameling van R-pakketten](https://www.tidyverse.org/packages) ontworpen om data science sneller, eenvoudiger en leuker te maken!

-   `tidymodels`: Het [tidymodels](https://www.tidymodels.org/) framework is een [verzameling van pakketten](https://www.tidymodels.org/packages/) voor modelleren en machine learning.

-   `janitor`: Het [janitor-pakket](https://github.com/sfirke/janitor) biedt eenvoudige hulpmiddelen voor het onderzoeken en opschonen van rommelige gegevens.

-   `corrplot`: Het [corrplot-pakket](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) biedt een visueel exploratie-instrument voor correlatiematrices dat automatische variabelenherordening ondersteunt om verborgen patronen tussen variabelen te helpen detecteren.

Je kunt ze installeren met:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Het onderstaande script controleert of je de pakketten hebt die nodig zijn om deze module te voltooien en installeert ze voor je als ze ontbreken.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

We zullen later deze geweldige pakketten laden en beschikbaar maken in onze huidige R-sessie. (Dit is slechts ter illustratie, `pacman::p_load()` heeft dat al voor je gedaan)

## 1. Een lineaire regressielijn

Zoals je hebt geleerd in Les 1, is het doel van een lineaire regressieoefening om een *lijn* *van* *beste* *fit* te kunnen plotten om:

-   **Variabele relaties te tonen**. De relatie tussen variabelen te laten zien.

-   **Voorspellingen te maken**. Nauwkeurige voorspellingen te maken over waar een nieuw datapunt zou vallen in relatie tot die lijn.

Om dit type lijn te tekenen, gebruiken we een statistische techniek genaamd **Least-Squares Regression**. De term `least-squares` betekent dat alle datapunten rondom de regressielijn worden gekwadrateerd en vervolgens opgeteld. Idealiter is die uiteindelijke som zo klein mogelijk, omdat we een laag aantal fouten willen, oftewel `least-squares`. De lijn van beste fit is dus de lijn die ons de laagste waarde geeft voor de som van de gekwadrateerde fouten - vandaar de naam *least squares regression*.

We doen dit omdat we een lijn willen modelleren die de kleinste cumulatieve afstand heeft tot al onze datapunten. We kwadrateren de termen voordat we ze optellen, omdat we ons bezighouden met de grootte ervan en niet met de richting.

> **🧮 Laat me de wiskunde zien**
>
> Deze lijn, genaamd de *lijn van beste fit*, kan worden uitgedrukt door [een vergelijking](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` is de '`verklarende variabele` of `voorspeller`'. `Y` is de '`afhankelijke variabele` of `uitkomst`'. De helling van de lijn is `b` en `a` is het snijpunt met de y-as, wat verwijst naar de waarde van `Y` wanneer `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "helling = $y/x$")
    Infographic door Jen Looper
>
> Bereken eerst de helling `b`.
>
> Met andere woorden, en verwijzend naar de oorspronkelijke vraag over onze pompoendata: "voorspel de prijs van een pompoen per schepel per maand", zou `X` verwijzen naar de prijs en `Y` naar de maand van verkoop.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infographic door Jen Looper
> 
> Bereken de waarde van Y. Als je ongeveer \$4 betaalt, moet het april zijn!
>
> De wiskunde die de lijn berekent, moet de helling van de lijn aantonen, die ook afhankelijk is van het snijpunt, of waar `Y` zich bevindt wanneer `X = 0`.
>
> Je kunt de methode van berekening voor deze waarden bekijken op de [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) website. Bezoek ook [deze Least-squares calculator](https://www.mathsisfun.com/data/least-squares-calculator.html) om te zien hoe de waarden van de getallen de lijn beïnvloeden.

Niet zo eng, toch? 🤓

#### Correlatie

Nog een term om te begrijpen is de **Correlatiecoëfficiënt** tussen gegeven X- en Y-variabelen. Met een spreidingsdiagram kun je deze coëfficiënt snel visualiseren. Een diagram met datapunten die netjes in een lijn liggen, heeft een hoge correlatie, maar een diagram met datapunten die overal tussen X en Y verspreid zijn, heeft een lage correlatie.

Een goed lineair regressiemodel is er een die een hoge (dichter bij 1 dan bij 0) Correlatiecoëfficiënt heeft met behulp van de Least-Squares Regression-methode met een regressielijn.


## **2. Een dans met data: een data frame maken voor modellering**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Kunstwerk door @allison_horst</figcaption>


<!--![Kunstwerk door \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Laad de benodigde bibliotheken en dataset. Zet de data om naar een data frame met een subset van de gegevens:

-   Selecteer alleen pompoenen die per schepel geprijsd zijn

-   Zet de datum om naar een maand

-   Bereken de prijs als een gemiddelde van de hoge en lage prijzen

-   Zet de prijs om zodat deze overeenkomt met de prijs per schepelhoeveelheid

> We hebben deze stappen behandeld in de [vorige les](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

In de geest van puur avontuur, laten we de [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) verkennen die eenvoudige functies biedt voor het onderzoeken en opschonen van rommelige gegevens. Laten we bijvoorbeeld eens kijken naar de kolomnamen van onze gegevens:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 We kunnen beter. Laten we deze kolomnamen `friendR` maken door ze om te zetten naar de [snake_case](https://en.wikipedia.org/wiki/Snake_case)-conventie met behulp van `janitor::clean_names`. Om meer te weten te komen over deze functie: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Veel tidyR 🧹! Nu, een dansje met de data met behulp van `dplyr`, zoals in de vorige les! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Goed gedaan!👌 Je hebt nu een schoon en overzichtelijk gegevensbestand waarop je je nieuwe regressiemodel kunt bouwen!

Zin in een spreidingsdiagram?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Een spreidingsdiagram herinnert ons eraan dat we alleen maandgegevens hebben van augustus tot en met december. We hebben waarschijnlijk meer gegevens nodig om conclusies te kunnen trekken op een lineaire manier.

Laten we onze modelleringsgegevens opnieuw bekijken:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Wat als we de `prijs` van een pompoen wilden voorspellen op basis van de kolommen `stad` of `pakket`, die van het type karakter zijn? Of nog eenvoudiger, hoe zouden we de correlatie kunnen vinden (wat vereist dat beide invoerwaarden numeriek zijn) tussen bijvoorbeeld `pakket` en `prijs`? 🤷🤷

Machine learning-modellen werken het beste met numerieke kenmerken in plaats van tekstwaarden, dus je moet categorische kenmerken meestal omzetten naar numerieke representaties.

Dit betekent dat we een manier moeten vinden om onze voorspellers opnieuw te formatteren, zodat een model ze effectiever kan gebruiken. Dit proces staat bekend als `feature engineering`.


## 3. Gegevens voorbereiden voor modellering met recepten 👩‍🍳👨‍🍳

Activiteiten die voorspellerwaarden herformatteren om ze gemakkelijker door een model te laten gebruiken, worden `feature engineering` genoemd.

Verschillende modellen hebben verschillende vereisten voor gegevensvoorbereiding. Bijvoorbeeld, least squares vereist `het coderen van categorische variabelen` zoals maand, variëteit en city_name. Dit houdt simpelweg in dat een kolom met `categorische waarden` wordt `vertaald` naar een of meer `numerieke kolommen` die de oorspronkelijke kolom vervangen.

Stel bijvoorbeeld dat je gegevens de volgende categorische eigenschap bevatten:

|  stad   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokio  |

Je kunt *ordinale codering* toepassen om elke categorie te vervangen door een unieke integerwaarde, zoals dit:

| stad |
|:----:|
|  0   |
|  1   |
|  2   |

En dat is precies wat we met onze gegevens gaan doen!

In deze sectie gaan we een ander geweldig Tidymodels-pakket verkennen: [recipes](https://tidymodels.github.io/recipes/) - dat is ontworpen om je te helpen je gegevens te verwerken **voordat** je je model traint. In de kern is een recept een object dat definieert welke stappen moeten worden toegepast op een dataset om deze klaar te maken voor modellering.

Laten we nu een recept maken dat onze gegevens voorbereidt voor modellering door een unieke integer te vervangen voor alle observaties in de voorspellerkolommen:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Geweldig! 👏 We hebben zojuist ons eerste recept gemaakt dat een uitkomst (prijs) specificeert en de bijbehorende voorspellers, waarbij alle voorspellerkolommen worden gecodeerd als een reeks gehele getallen 🙌! Laten we het snel opsplitsen:

-   De oproep naar `recipe()` met een formule vertelt het recept de *rollen* van de variabelen, waarbij gebruik wordt gemaakt van de gegevens uit `new_pumpkins` als referentie. Bijvoorbeeld, de kolom `price` is toegewezen aan de rol `outcome`, terwijl de rest van de kolommen de rol `predictor` hebben gekregen.

-   `step_integer(all_predictors(), zero_based = TRUE)` specificeert dat alle voorspellers moeten worden omgezet in een reeks gehele getallen, waarbij de nummering begint bij 0.

We begrijpen dat je misschien denkt: "Dit is zo gaaf!! Maar wat als ik wil bevestigen dat de recepten precies doen wat ik verwacht? 🤔"

Dat is een geweldige gedachte! Zie je, zodra je recept is gedefinieerd, kun je de parameters schatten die nodig zijn om de gegevens daadwerkelijk voor te verwerken, en vervolgens de verwerkte gegevens extraheren. Dit is meestal niet nodig wanneer je Tidymodels gebruikt (we zullen zo meteen de normale conventie zien -> `workflows`), maar het kan handig zijn als je een soort controle wilt uitvoeren om te bevestigen dat de recepten doen wat je verwacht.

Hiervoor heb je twee extra werkwoorden nodig: `prep()` en `bake()`. En zoals altijd helpen onze kleine R-vrienden van [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) je om dit beter te begrijpen!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Illustratie door @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): schat de benodigde parameters op basis van een trainingsset, die later kunnen worden toegepast op andere datasets. Bijvoorbeeld, voor een bepaalde voorspellerkolom, welke observatie krijgt het gehele getal 0, 1, 2, enzovoort toegewezen.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): neemt een voorbereide recipe en past de bewerkingen toe op elke dataset.

Dat gezegd hebbende, laten we onze recepten voorbereiden en toepassen om echt te bevestigen dat, achter de schermen, de voorspellerkolommen eerst worden gecodeerd voordat een model wordt getraind.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 De verwerkte data `baked_pumpkins` heeft alle voorspellers gecodeerd, wat bevestigt dat de preprocessiestappen die we als ons recept hebben gedefinieerd, inderdaad werken zoals verwacht. Dit maakt het misschien moeilijker voor jou om te lezen, maar veel begrijpelijker voor Tidymodels! Neem even de tijd om uit te zoeken welke observatie is gekoppeld aan welk overeenkomstig geheel getal.

Het is ook goed om te vermelden dat `baked_pumpkins` een data frame is waarop we berekeningen kunnen uitvoeren.

Bijvoorbeeld, laten we proberen een goede correlatie te vinden tussen twee punten in je data om mogelijk een goed voorspellend model te bouwen. We gebruiken hiervoor de functie `cor()`. Typ `?cor()` om meer te weten te komen over deze functie.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Zoals blijkt, is er slechts een zwakke correlatie tussen de Stad en de Prijs. Er is echter een iets betere correlatie tussen het Pakket en de Prijs. Dat is logisch, toch? Meestal geldt: hoe groter de productdoos, hoe hoger de prijs.

Laten we, nu we toch bezig zijn, ook proberen een correlatiematrix van alle kolommen te visualiseren met behulp van het `corrplot`-pakket.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Veel beter.

Een goede vraag om nu aan deze gegevens te stellen is: '`Welke prijs kan ik verwachten voor een bepaald pompoenpakket?`' Laten we meteen aan de slag gaan!

> Opmerking: Wanneer je **`bake()`** toepast op het voorbereide recept **`pumpkins_prep`** met **`new_data = NULL`**, haal je de verwerkte (d.w.z. gecodeerde) trainingsgegevens op. Als je een andere dataset had, bijvoorbeeld een testset, en wilde zien hoe een recept deze zou voorbewerken, zou je simpelweg **`pumpkins_prep`** bakken met **`new_data = test_set`**.

## 4. Bouw een lineair regressiemodel

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographic door Dasani Madipalli</figcaption>


Nu we een recept hebben gemaakt en daadwerkelijk hebben bevestigd dat de gegevens correct worden voorbewerkt, gaan we nu een regressiemodel bouwen om de volgende vraag te beantwoorden: `Welke prijs kan ik verwachten voor een bepaald pompoenpakket?`

#### Train een lineair regressiemodel met behulp van de trainingsset

Zoals je waarschijnlijk al hebt begrepen, is de kolom *price* de `uitkomst`-variabele, terwijl de kolom *package* de `voorspeller`-variabele is.

Om dit te doen, splitsen we eerst de gegevens zodat 80% in de trainingsset terechtkomt en 20% in de testset. Vervolgens definiëren we een recept dat de voorspellerkolom omzet in een reeks gehele getallen, en daarna bouwen we een modelspecificatie. We zullen ons recept niet voorbereiden en bakken, omdat we al weten dat het de gegevens naar verwachting zal voorbewerken.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Goed gedaan! Nu we een recept en een modelspecificatie hebben, moeten we een manier vinden om deze samen te voegen in een object dat eerst de gegevens zal voorbewerken (prep+bake achter de schermen), het model zal trainen op de voorbewerkte gegevens en ook ruimte biedt voor eventuele nabewerkingsactiviteiten. Klinkt dat niet geruststellend!🤩

In Tidymodels wordt dit handige object een [`workflow`](https://workflows.tidymodels.org/) genoemd, en het bevat op een handige manier je modelcomponenten! Dit is wat we in *Python* *pipelines* zouden noemen.

Laten we dus alles samenvoegen in een workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Bovendien kan een workflow op vrijwel dezelfde manier worden aangepast/getraind als een model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Uit de modeloutput kunnen we de coëfficiënten zien die tijdens de training zijn geleerd. Ze vertegenwoordigen de coëfficiënten van de lijn van beste fit die ons de laagste totale fout geeft tussen de werkelijke en voorspelde variabele.

#### Modelprestaties evalueren met behulp van de testset

Het is tijd om te zien hoe het model heeft gepresteerd 📏! Hoe doen we dit?

Nu we het model hebben getraind, kunnen we het gebruiken om voorspellingen te maken voor de test_set met `parsnip::predict()`. Vervolgens kunnen we deze voorspellingen vergelijken met de werkelijke labelwaarden om te beoordelen hoe goed (of niet!) het model werkt.

Laten we beginnen met het maken van voorspellingen voor de testset en vervolgens de kolommen aan de testset koppelen.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Ja, je hebt net een model getraind en gebruikt om voorspellingen te maken! 🔮 Is het goed? Laten we de prestaties van het model evalueren!

In Tidymodels doen we dit met behulp van `yardstick::metrics()`! Voor lineaire regressie richten we ons op de volgende metrics:

-   `Root Mean Square Error (RMSE)`: De vierkantswortel van de [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Dit levert een absolute maatstaf op in dezelfde eenheid als het label (in dit geval de prijs van een pompoen). Hoe kleiner de waarde, hoe beter het model (simpel gezegd vertegenwoordigt het de gemiddelde prijs waarmee de voorspellingen fout zijn!).

-   `Coefficient of Determination (meestal bekend als R-squared of R2)`: Een relatieve maatstaf waarbij een hogere waarde betekent dat het model beter past. In essentie vertegenwoordigt deze metric hoeveel van de variatie tussen voorspelde en werkelijke labelwaarden het model kan verklaren.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Daar gaat de modelprestaties. Laten we kijken of we een betere indicatie kunnen krijgen door een spreidingsdiagram van het pakket en de prijs te visualiseren en vervolgens de voorspellingen te gebruiken om een lijn van beste fit te overlayen.

Dit betekent dat we de testset moeten voorbereiden en verwerken om de pakketkolom te coderen en deze vervolgens te koppelen aan de voorspellingen die door ons model zijn gemaakt.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Geweldig! Zoals je kunt zien, generaliseert het lineaire regressiemodel de relatie tussen een pakket en de bijbehorende prijs niet echt goed.

🎃 Gefeliciteerd, je hebt zojuist een model gemaakt dat kan helpen de prijs van een paar soorten pompoenen te voorspellen. Jouw pompoenveld voor de feestdagen zal prachtig zijn. Maar je kunt waarschijnlijk een beter model maken!

## 5. Bouw een polynomiaal regressiemodel

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographic door Dasani Madipalli</figcaption>


<!--![Infographic door Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Soms heeft onze data geen lineaire relatie, maar willen we toch een uitkomst voorspellen. Polynomiale regressie kan ons helpen voorspellingen te doen voor meer complexe niet-lineaire relaties.

Neem bijvoorbeeld de relatie tussen het pakket en de prijs in onze pompoen-dataset. Hoewel er soms een lineaire relatie is tussen variabelen - hoe groter de pompoen qua volume, hoe hoger de prijs - kunnen deze relaties soms niet worden weergegeven als een vlak of rechte lijn.

> ✅ Hier zijn [nog wat voorbeelden](https://online.stat.psu.edu/stat501/lesson/9/9.8) van data die polynomiale regressie kunnen gebruiken
>
> Kijk nog eens naar de relatie tussen Variëteit en Prijs in de vorige grafiek. Lijkt deze spreidingsplot noodzakelijkerwijs geanalyseerd te moeten worden met een rechte lijn? Misschien niet. In dit geval kun je polynomiale regressie proberen.
>
> ✅ Polynomen zijn wiskundige uitdrukkingen die kunnen bestaan uit één of meer variabelen en coëfficiënten

#### Train een polynomiaal regressiemodel met behulp van de trainingsset

Polynomiale regressie creëert een *gebogen lijn* om niet-lineaire data beter te modelleren.

Laten we kijken of een polynomiaal model beter presteert bij het maken van voorspellingen. We volgen een procedure die enigszins lijkt op wat we eerder hebben gedaan:

-   Maak een recept dat de preprocessing-stappen specificeert die op onze data moeten worden uitgevoerd om deze klaar te maken voor modellering, zoals: het coderen van voorspellers en het berekenen van polynomen van graad *n*

-   Bouw een modelspecificatie

-   Bundel het recept en de modelspecificatie in een workflow

-   Maak een model door de workflow te fitten

-   Evalueer hoe goed het model presteert op de testdata

Laten we aan de slag gaan!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Evalueer de prestaties van het model

👏👏Je hebt een polynomiaal model gebouwd, laten we voorspellingen maken op de testset!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, laten we evalueren hoe het model presteerde op de test_set met behulp van `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Veel betere prestaties.

De `rmse` is gedaald van ongeveer 7 naar ongeveer 3, wat aangeeft dat de fout tussen de werkelijke prijs en de voorspelde prijs is verminderd. Je kunt dit *losjes* interpreteren als dat verkeerde voorspellingen gemiddeld ongeveer \$3 fout zijn. De `rsq` is gestegen van ongeveer 0,4 naar 0,8.

Al deze statistieken geven aan dat het polynomiale model veel beter presteert dan het lineaire model. Goed gedaan!

Laten we kijken of we dit kunnen visualiseren!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Je kunt een gebogen lijn zien die beter bij je gegevens past! 🤩

Je kunt dit nog vloeiender maken door een polynomiale formule door te geven aan `geom_smooth`, zoals dit:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Net zoals een vloeiende curve!🤩

Hier is hoe je een nieuwe voorspelling zou maken:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


De voorspelling van het `polynomial model` is logisch, gezien de spreidingsdiagrammen van `price` en `package`! En als dit een beter model is dan het vorige, kijkend naar dezelfde gegevens, moet je rekening houden met deze duurdere pompoenen!

🏆 Goed gedaan! Je hebt twee regressiemodellen gemaakt in één les. In het laatste deel over regressie leer je over logistische regressie om categorieën te bepalen.

## **🚀Uitdaging**

Test verschillende variabelen in dit notebook om te zien hoe correlatie overeenkomt met de nauwkeurigheid van het model.

## [**Quiz na de les**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Review & Zelfstudie**

In deze les hebben we geleerd over Lineaire Regressie. Er zijn andere belangrijke soorten regressie. Lees meer over Stepwise, Ridge, Lasso en Elasticnet technieken. Een goede cursus om meer te leren is de [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Als je meer wilt leren over het gebruik van het geweldige Tidymodels-framework, bekijk dan de volgende bronnen:

-   Tidymodels website: [Aan de slag met Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn en Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **BEDANKT AAN:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) voor het maken van de geweldige illustraties die R toegankelijker en aantrekkelijker maken. Vind meer illustraties in haar [galerij](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
